# 🌐 Scraping de Dados de Notícias Financeiras com Selenium

Este notebook tem como objetivo **extrair dados de notícias financeiras** do site Investing.com utilizando a biblioteca Selenium. O processo inclui a extração de elementos HTML através de seletores CSS e tags, e a captura de conteúdo relevante para análise posterior.

1. **Configuração do Selenium**: Configuração do driver do Chrome para interação com a página web.
2. **Extração de Dados**:
   - Através de **tags HTML**: Extração de todos os elementos de uma tag específica.
   - Através de **seletor CSS incremental**: Extração de elementos que seguem um padrão incremental no nome do seletor CSS.
   - Captura de conteúdo de elementos específicos na página, como notícias financeiras.

🚀 **Objetivo final**: Obter e organizar dados de notícias financeiras para análise ou modelagem posterior.

In [1]:
# !python -m pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [3]:
driver = webdriver.Chrome()

driver.get("https://www.investing.com/")
# print(driver.title)

In [4]:
def print_all_elements_by_tag(tag_name: str):
    """
    Extrai e imprime o texto de todos os elementos de uma página que possuem a tag especificada.

    Args:
        tag_name (str): Nome da tag HTML a ser buscada (por exemplo, 'div', 'p', 'span').

    Retorna:
        None: A função apenas imprime os textos dos elementos encontrados.

    Exemplo de uso:
        print_all_elements_by_tag('p')  # Imprime o texto de todos os elementos <p> na página.
    """
    elem = driver.find_elements(By.TAG_NAME, tag_name)

    for e in elem:
        text = e.text
        if text:
            print(text)

In [5]:
def get_elements_by_tag(driver_, parent_element, tag_type: str):
    """
    Extrai o texto de elementos filhos de uma tag específica dentro de um elemento pai.

    Args:
        driver_ (WebDriver): Instância do WebDriver.
        parent_element (WebElement): O elemento pai onde a busca pelos filhos será realizada.
        tag_type (str): Nome da tag HTML dos elementos filhos a serem encontrados (por exemplo, 'a', 'div').

    Retorna:
        list: Uma lista contendo os textos dos elementos filhos encontrados e, se existirem, suas sublistas de filhos.

    Exemplo de uso:
        get_elements_by_tag(driver, parent_element, 'div')  # Extrai os textos de todas as divs dentro de parent_element.
    """
    elements = []
    
    # Encontre todos os elementos filhos do tipo especificado dentro do elemento pai
    child_elements = parent_element.find_elements(By.TAG_NAME, tag_type)
    
    for element in child_elements:
        # Capture o conteúdo (texto) do elemento
        element_content = element.text
        
        # Verifique se o elemento atual tem filhos do mesmo tipo
        child_structure = get_elements_by_tag(driver_, element, tag_type)
        
        # Se houver filhos, adicione-os como sublista; caso contrário, adicione apenas o conteúdo
        if child_structure:
            elements.append([element_content, child_structure])
        else:
            elements.append(element_content)
    
    return elements

In [6]:
def get_elements_by_incremental_css_name(css_selector_template: str, start: int, end: int):
    """
    Extrai o texto de elementos na página com um seletor CSS incremental, baseado em um template de seletor.

    Args:
        css_selector_template (str): Template do seletor CSS com um espaço reservado para um número (por exemplo, '[data-test="homepage-news-list-item-{}"]').
        start (int): Número inicial para começar a busca.
        end (int): Número final para terminar a busca.

    Retorna:
        list: Uma lista contendo os textos dos elementos encontrados dentro do intervalo especificado.

    Exemplo de uso:
        get_elements_by_incremental_css_name('[data-test="homepage-news-list-item-{}"]', 1, 3)  # Extrai os textos dos elementos 1, 2 e 3.
    """
    # Array para armazenar os textos dos elementos
    elements_texts = []

    # Iterar sobre o intervalo de números (start até end)
    for i in range(start, end + 1):
        # Criar o seletor CSS com o número atual
        css_selector = css_selector_template.format(i)
        try:
            # Tentar encontrar o elemento
            element = driver.find_element(By.CSS_SELECTOR, css_selector)
            # Adicionar o texto do elemento ao array
            elements_texts.append(element.text)
        except:
            # Se o elemento não for encontrado, ignorar e continuar
            print(f"Elemento {css_selector} não encontrado.")
    
    return elements_texts

In [7]:
element = driver.find_element(By.CSS_SELECTOR, '[data-test="homepage-news-main-item-content"]')
element.text

'China wants to make a deal ’very badly’, says Trump\nInvesting.com -- During a recent White House event, President Donald Trump made several noteworthy announcements. He revealed that Washington D.C. will host the 2027 NFL draft and he expressed intent to aid the film industry, which he claimed is being ’decimated’ by other countries. The President, w...'

In [8]:
elements_texts = get_elements_by_incremental_css_name('[data-test="homepage-news-list-item-{}"]', 1, 3)
elements_texts

['Ford shares fall as tariff concerns overshadow earnings beat',
 'Partner News\nActTrader at 25: A Legacy Platform with the Craftsmanship of a Watchmaker and the Energy of a Startup',
 'Dollar slides against peers, weighed down by fresh tariff worries']